# --- Day 7: The Sum of Its Parts ---

In [1]:
with open('input/day7.txt') as f:
    instructions = f.readlines()
print(instructions[0])

Step S must be finished before step V can begin.



In [2]:
import re
from collections import defaultdict

In [3]:
def build_pairs(instructions):
    inst = defaultdict(list)
    for i in instructions:
        pairs = re.findall(r'\s([A-Z])\s', i)
        inst[pairs[1]].append(pairs[0])
    return(inst)

In [4]:
# list of all steps:
def get_steps(inst):
    l = list(inst.keys())
    for v in inst.values():
        l.extend(v)
    return set(l)

In [5]:
def build_sequence(steps, inst, done):
    for s in sorted(steps):
        if s in done:
            pass
        else:
            if not s in inst.keys():
                done.append(s)
                build_sequence(steps, inst, done)
            else:
                test = [c for c in inst[s] if c in done]
                if test == inst[s]:
                    done.append(s)
                    build_sequence(steps, inst, done)
                else:
                    pass
    return ''.join(done)

In [6]:
test = ['Step C must be finished before step A can begin.',
        'Step C must be finished before step F can begin.',
        'Step A must be finished before step B can begin.',
        'Step A must be finished before step D can begin.',
        'Step B must be finished before step E can begin.',
        'Step D must be finished before step E can begin.',
        'Step F must be finished before step E can begin.']

In [13]:
# test
'''
inst = build_pairs(test)
steps = get_steps(inst)
'''
print('not testing...')

not testing...


In [14]:
inst = build_pairs(instructions)
steps = get_steps(inst)
done = []

In [15]:
build_sequence(steps, inst, done=[])

'JNOIKSYABEQRUVWXGTZFDMHLPC'

## --- Part Two ---

In [16]:
import pandas as pd

In [17]:
import string
def create_duration(time=60):
    dur = {}
    for i, l in enumerate(string.ascii_uppercase):
        dur[l] = i+time+1
    return dur

In [18]:
def ready(steps, inst, done):
    ready = ''
    for s in steps:
        conditions = [c for c in inst[s] if c in done] == inst[s]
        if (not s in done) & conditions:
            ready += s
        else:
            pass
    return ''.join(sorted(ready))    

In [19]:
def initiate_workers(nr):
    wrkrs = []
    for i in range(nr):
        wrkrs.append((i,'',0))
    return pd.DataFrame(data=wrkrs, columns=['worker','step','time_rem'])

In [20]:
def time_sequence(allsteps, inst, duration, done, workers, elapsed):
    available_workers = workers[workers['time_rem']==0]
    steps_ready = ready(allsteps, inst, done)
    assign = min(len(available_workers), len(steps_ready))
    if assign > 0:
        for i in range(assign):
            worker = available_workers.iloc[i,0]
            workers.loc[worker, 'step'] = steps_ready[i]
            workers.loc[worker, 'time_rem'] = duration[steps_ready[i]]
            allsteps.remove(steps_ready[i])
    else:
        pass
    
    working = workers[workers['time_rem']>0] # anybody working?

    if len(working) > 0:
        worker = working[working['time_rem']==min(working['time_rem'])]['worker']
        elapse = workers.loc[worker, 'time_rem'].item()
        elapsed += elapse
        done += workers.loc[worker, 'step'].item()
        workers.loc[worker, 'step'] = ''
        workers['time_rem'] = workers['time_rem'].apply(lambda x: max(x-elapse, 0))
    else:
        pass
    
    if len(allsteps) == 0:
        print('done = {}'.format(done))
        print('elapsed time = {}'.format(elapsed))
        return done, elapsed
    else:        
        return time_sequence(allsteps, inst, duration, done, workers, elapsed)


In [21]:
inst = build_pairs(test)
steps = get_steps(inst)
dur = create_duration(time=0)
d, e = time_sequence(steps.copy(), inst, dur, '', initiate_workers(2), 0)

done = CABFDE
elapsed time = 15


In [22]:
inst = build_pairs(instructions)
steps = get_steps(inst)
dur = create_duration()
d, e = time_sequence(steps, inst, dur, '', initiate_workers(5), 0)

done = JNOSIKYAQUWBREVXGTZFMDHLPC
elapsed time = 1099
